In [ ]:
# add package to path if not installed in editable mode
import sys, pathlib
repo_root = pathlib.Path.cwd()  # set to your repo root if needed
sys.path.append(str(repo_root / "src"))

import numpy as np
import pandas as pd

from optimal_ipr.distributions import value_distribution, build_theta_distribution
from optimal_ipr.probability import build_subjective_probability
from optimal_ipr.cost import build_cost_function
from optimal_ipr.fee import build_fee_schedule
from optimal_ipr.lookup import build_lookup_tables
from optimal_ipr.outcomes import welfare_outcomes

# value distribution
v_grid, v_weights = value_distribution(n_v=51, sigma=3.0)

# theta distribution
f, F, F_inv = build_theta_distribution(noise_level=0.10)

# scalar wrapper for F when a scalar is required
F_scalar = lambda t: float(F(np.array([t])))

p = build_subjective_probability(base_k=1.25, m_comp=5, F=F, F_inv=F_inv)
c = build_cost_function(f, TARGET_AVG_COST_SHARE=0.50, C_MIN_COST=0.05, GAMMA_C_COST=3.0)
Z = build_fee_schedule(zeta=0.04, fee_M=1.5)

tau_d_grid = np.array([0.20, 0.25, 0.30])   # adjust as needed
tau_f_grid = np.array([0.00, 0.05])         # adjust as needed
bar_beta_grid = np.linspace(0.0, 1.0, 101)  # matches earlier modules

theta_tilde_table, theta_winner_table, get_tau_d_index, get_tau_f_index, get_bar_beta_index, get_v_index = \
    build_lookup_tables(p, c, Z, f, F_scalar, tau_d_grid, tau_f_grid, bar_beta_grid, v_grid)

# government weights over theta
gov_prefs = {
    "utilitarian": (lambda theta: 1.0),
}

# regulator schemes
reg_prefs = {
    "utilitarian": "special_case",            # matches RegulatorModel's special-case handling
    # add others if desired, e.g.: "weighted": {"phi": 0.5, "psi": 0.2}
}

results_table = welfare_outcomes(
    tau_d=tau_d_grid,
    tau_f=tau_f_grid,
    gov_prefs=gov_prefs,
    reg_prefs=reg_prefs,
    v_grid=v_grid,
    v_weights=v_weights,
    theta_tilde_table=theta_tilde_table,
    theta_winner_table=theta_winner_table,
    f=f,
    F=F,            # vectorized is fine; module wraps when needed
    F_inv=F_inv,
    p=p,
    c=c,
    Z=Z,
    feas=True,      # enforce feasibility
)

results_table
